In [59]:
import pandas as pd
import numpy as np
import os

In [60]:
def get_sample_df(sample_file):
    sample = pd.read_csv(sample_file, encoding='iso-8859-1', nrows=1)
    return pd.DataFrame(columns=sample.columns)

In [61]:
# Chuẩn bị thông tin cho việc khai báo

# Thông tin quy cách theo từng cặp SKU-Store
def prepare_minor(file_dir, sheet_name):
    """
    This function is designed to read the sheet containing packing information of each SKU at each store, then clean and transform it into
    a dataframe which has the structure of keys.
    
    Input: 
        - The path of of the xlsx template files 
        - Sheet name containing information of packing quantity per piece.
    
    Output: Dataframe of packing information, primary key = (SKu + Store)

    """
    quy_cach = pd.read_excel(file_dir,sheet_name= sheet_name,header=2)
    col_rename = quy_cach.loc[0].to_list()[:3]
    for i, col in enumerate(quy_cach.columns):
        if i == 3:
            break
        quy_cach.rename(columns={col: col_rename[i]},inplace=True)
    quy_cach.drop(index=0, columns=['Unnamed: 3', 'STORE'],inplace=True)

    quy_cach_melt = pd.melt(quy_cach, id_vars=quy_cach.columns[:3], 
                            value_vars = quy_cach.columns[3:],
                            var_name='Store',
                            value_name='Quy Cach')
    return quy_cach_melt
quy_cach = prepare_minor("FFS_Template (R).xlsx", sheet_name='Sheet1')
quy_cach = quy_cach[~quy_cach['Store'].isin([2173,799])]

# Thông tin chung của SKU
def prepare_sku_infor(file_dir, sheet_name):
    sku_infor = pd.read_excel("FFS_Template (R).xlsx", sheet_name='Danh mục sku')
    for col in sku_infor.columns:
        sku_infor.rename(columns={col: str(col).replace('\n','')},inplace=True)
    return sku_infor.iloc[:,:6]
sku_infor = prepare_sku_infor("FFS_Template (R).xlsx", sheet_name='Danh mục sku')

# Thông tin network
def prepare_network(file_dir, sheet_name):
    network = pd.read_excel(file_dir, sheet_name= sheet_name)
    network.rename(columns={'thời gian yêu cầu có đơn hàng trước\nmấy ngày.3': 'LeadTime',
                            'thời gian lock.4': 'LockTime'},inplace=True)
    return network[['Store','LeadTime','LockTime']].drop(index=0)
network_infor = prepare_network("FFS_Template (R).xlsx", 'Sheet2')

# Dữ liệu Item đã khai báo trên D&F
item_traicay = pd.read_csv('item_traicay.csv', encoding='iso-8859-1')

# Dữ liệu Loc đã khai báo trên D&F
all_loc = pd.read_csv('DF_All_Loc.csv', encoding='iso-8859-1')

In [62]:
df_khai_bao = get_sample_df('sourcing_sample.csv')


# Xác định các cặp *Item *Dest từ file quy cách
df_khai_bao['*Item'] = quy_cach['SKU']
df_khai_bao['*Vendor'] = quy_cach['Mã NCC']
df_khai_bao['*Dest'] = quy_cach['Store']
df_khai_bao['QC Dong Goi lam tron'] = quy_cach['Quy Cach']
df_khai_bao['QC Dong Goi toi thieu'] = quy_cach['Quy Cach']

# Xác định các thông tin liên quan đến SKU
df_khai_bao['UDC_SEQ_ITEM'] =  pd.merge(df_khai_bao[['*Item']], sku_infor[['Mã SKU','Số thứ tự SKU theo NCC']], 
                                                            how='left', 
                                                            left_on='*Item',
                                                            right_on='Mã SKU')['Số thứ tự SKU theo NCC'].tolist()
df_khai_bao['UDC_SEQ_VENDOR'] =  pd.merge(df_khai_bao[['*Item']], sku_infor[['Mã SKU','Số thứ tự NCC trên đơn hàng']], 
                                                            how='left', 
                                                            left_on='*Item',
                                                            right_on='Mã SKU')['Số thứ tự NCC trên đơn hàng'].tolist()

# Xác định các thông tin liên quan đến network
df_khai_bao['TransLeadTime'] = pd.merge(df_khai_bao[['*Dest']], network_infor[['Store','LeadTime']], 
                                                            how='left', 
                                                            left_on='*Dest',
                                                            right_on='Store')['LeadTime'].tolist()

# Thông số mặc định
df_khai_bao['UDC_TPTS_LOCK'].fillna('FALSE',inplace=True) 
df_khai_bao['ONOFF'].fillna('FALSE',inplace=True) 

In [63]:
df_khai_bao.rename(columns={'Descr.1': 'Descr'}).to_csv('sourcing_trai-cay_080424.csv', index=False)